[//]: # ( Plant Card Generator based on public and free API queries. )

[//]: # ( Notebook language is Python 3 and uses requests and pandas libraries. )

[//]: # ( This notebook is designed to create plant care cards by fetching data from various APIs. )

[//]: # ( The generated cards will include information such as plant name, care instructions, and images.)

[//]: # ( # License: MIT License )

[//]: # ( Date: 2024-06-15 )

[//]: # ( Updated: 2025-11-04 )

[//]: # ( Creator: Pekka Sihvonen )

[//]: # ( Version: 1.2 - Multilingual Support )

# 🌲 Plant Card Generator
**Version 1.2** | Updated: 2025-11-04 | **🌍 Now with Multilingual Support!**

## 📋 Overview

This notebook creates scientifically documented **plant cards** by aggregating data from multiple open data sources. The generator is designed for researchers, students, and gardening enthusiasts who want to collect reliable botanical information in one place.

**✨ NEW in v1.2:** Wikipedia summaries and AI-generated descriptions can now be generated in **9 languages**!

### 🌍 Supported Languages
- 🇬🇧 English
- 🇫🇮 Finnish (Suomi)
- 🇸🇪 Swedish (Svenska)
- 🇩🇪 German (Deutsch)
- 🇫🇷 French (Français)
- 🇪🇸 Spanish (Español)
- 🇮🇹 Italian (Italiano)
- 🇯🇵 Japanese (日本語)
- 🇨🇳 Chinese (中文)

### 🎯 Use Cases
- Botanical research and education (in your language!)
- Garden planning and plant selection
- Biodiversity documentation
- Plant care guide creation
- Language learning with botanical content

### 📊 Data Sources

This notebook utilizes the following reliable databases:

| Data Source | Type | API Key | Information |
|------------|------|---------|-------------|
| **GBIF** | Open | Not required | Taxonomy, distribution, occurrences |
| **Trefle** | Open | Required | Growth characteristics, images |
| **Wikimedia Commons** | Open | Not required | Botanical illustrations |
| **EOL** | Open | Not required | Ecological data |
| **Wikipedia** | Open | Not required | General descriptions (multilingual!) |
| **iNaturalist** | Open | Not required | Community observations |
| **Laji.fi** | Open | Required | Finnish species names |
| **BHL** | Open | Required | Historical illustrations |
| **Google Gemini** | AI | Required | Summary generation (multilingual!) |

### 🔑 Obtaining API Keys

Links to API services:

- [Gemini API](https://aistudio.google.com) — For AI summaries (in any supported language!)
- [Trefle API](https://trefle.com) — For growth characteristics and images
- [Laji.fi API](https://laji.fi/en/about/13) — For Finnish species names (optional)
- [BHL API](https://www.biodiversitylibrary.org/api2/docs/) — For historical illustrations (optional)

### ⚠️ Important Notes
- Data is fetched in real-time, so availability may vary
- API rate limits may restrict query volume
- Always verify information from primary sources before scientific use
- AI-generated descriptions are indicative, not peer-reviewed
- Language selection affects Wikipedia and AI content only (UI remains in English)

## 📚 Data Source Descriptions

### Free Sources (No API Key Required)

- **GBIF (Global Biodiversity Information Facility)** — World's largest biodiversity database. Provides taxonomy, distribution areas, and occurrence records.
  - Source: https://www.gbif.org
  - License: CC0 / CC-BY
  
- **Wikipedia REST API** — General descriptions and summaries in multiple languages. NOTE: Not peer-reviewed scientific information.
  - Source: https://www.wikipedia.org
  - License: CC-BY-SA
  
- **Wikimedia Commons** — Free botanical illustrations and historical plant drawings.
  - Source: https://commons.wikimedia.org
  - License: Varies by image (mostly CC-BY or Public Domain)
  
- **iNaturalist API** — Community-based observation service. Provides observation counts, common names, and photographs.
  - Source: https://www.inaturalist.org
  - License: Varies by user
  
- **EOL (Encyclopedia of Life)** — Ecological data, habitats, and reproduction.
  - Source: https://eol.org
  - License: Varies by content

### Sources Requiring API Keys

- **Trefle** — Comprehensive plant database with growth characteristics (pH, growth form, etc.)
- **BHL (Biodiversity Heritage Library)** — Historical library with botanical illustrations
- **Google Gemini** — AI model for generating summaries from collected data

## ⚙️ Step 1: Installation and Configuration

### Instructions

1. **Install libraries** — Run the code below to install required Python libraries
2. **Add API keys** — Use Colab's **Secrets** feature (key icon on the left)
   - `GOOGLE_API_KEY` — Gemini API
   - `TREFLE_API_KEY` — Trefle API
   - `BHL_API_KEY` — BHL API (optional)
   - `LAJI_TOKEN` — Laji.fi API (optional, for Finnish names)
3. **Select output language** — Choose language for generated content (Wikipedia summaries, AI descriptions)
4. **Enter plant name** — Use scientific name (e.g., *Quercus robur*)
5. **Select data sources** — Check your desired data sources
6. **Save settings** — Press the green button

### 💡 Tips
- Always use scientific names (Latin binomial)
- Check spelling (e.g., verify with GBIF search)
- API keys are stored securely in Colab Secrets
- Language selection affects Wikipedia and AI-generated content only

In [ ]:
# ============================================================================
# STEP 1.1: Library Installation and Import
# ============================================================================
"""
Installs required Python libraries and imports them.
This cell should be run first.
"""

# Installation (run once)
!pip install -q requests pandas google-generativeai ipywidgets

# Library imports
import warnings
from datetime import datetime
from typing import Any, Dict, List, Optional

import google.generativeai as genai
import ipywidgets as widgets
import pandas as pd
import requests
from IPython.display import Image, Markdown, display

# Colab-specific import
try:
    from google.colab import userdata

    COLAB_ENV = True
    print("✅ Google Colab environment detected")
except ImportError:
    COLAB_ENV = False
    print("⚠️ Not in Colab environment. API keys must be set manually.")
    # Create mock userdata class for local use

    class MockUserData:
        def get(self, key: str) -> str:
            import os

            return os.environ.get(key, "")

    userdata = MockUserData()

# Reduce warnings
warnings.filterwarnings("ignore")

print(f"📅 Run at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ Libraries imported successfully\n")

# ============================================================================
# STEP 1.2: User Interface and Settings
# ============================================================================
"""
Creates an interactive user interface for plant name entry, language selection,
and data source selection.
"""

# Global variables
plant_name: str = ""
output_language: str = "en"
TREFLE_API_KEY: str = ""
GEMINI_API_KEY: str = ""
BHL_API_KEY: str = ""
LAJI_TOKEN: str = ""
API_SETTINGS: Dict[str, bool] = {}

# UI components
plant_name_widget = widgets.Text(
    value=globals().get("plant_name", ""),
    placeholder="E.g., Quercus robur (use scientific name)",
    description="🌿 Plant:",
    disabled=False,
    layout=widgets.Layout(width="500px"),
    style={"description_width": "100px"},
)

# Language selector for output content
language_widget = widgets.Dropdown(
    options=[
        ("🇬🇧 English", "en"),
        ("🇫🇮 Finnish (Suomi)", "fi"),
        ("🇸🇪 Swedish (Svenska)", "sv"),
        ("🇩🇪 German (Deutsch)", "de"),
        ("🇫🇷 French (Français)", "fr"),
        ("🇪🇸 Spanish (Español)", "es"),
        ("🇮🇹 Italian (Italiano)", "it"),
        ("🇯🇵 Japanese (日本語)", "ja"),
        ("🇨🇳 Chinese (中文)", "zh"),
    ],
    value=globals().get("output_language", "en"),
    description="🌍 Language:",
    style={"description_width": "100px"},
    layout=widgets.Layout(width="500px"),
)

# Default settings
default_settings = globals().get(
    "API_SETTINGS",
    {
        "gbif": True,
        "trefle": True,
        "eol": True,
        "gbif_occ": True,
        "gbif_dist": True,
        "wikipedia": True,
        "inaturalist": True,
        "wikimedia": True,
        "laji": False,
        "bhl": False,
        "gemini": True,
    },
)

# Checkboxes for data sources
use_gbif_widget = widgets.Checkbox(
    value=default_settings.get("gbif", True),
    description="🌍 GBIF taxonomy",
    style={"description_width": "200px"},
)

use_trefle_widget = widgets.Checkbox(
    value=default_settings.get("trefle", True),
    description="🌱 Trefle characteristics",
    style={"description_width": "200px"},
)

use_eol_widget = widgets.Checkbox(
    value=default_settings.get("eol", True),
    description="🔬 EOL ecological data",
    style={"description_width": "200px"},
)

use_gbif_occ_widget = widgets.Checkbox(
    value=default_settings.get("gbif_occ", True),
    description="📊 GBIF occurrences",
    style={"description_width": "200px"},
)

use_gbif_dist_widget = widgets.Checkbox(
    value=default_settings.get("gbif_dist", True),
    description="🗺️ GBIF distribution",
    style={"description_width": "200px"},
)

use_wikipedia_widget = widgets.Checkbox(
    value=default_settings.get("wikipedia", True),
    description="📖 Wikipedia summary",
    style={"description_width": "200px"},
)

use_inaturalist_widget = widgets.Checkbox(
    value=default_settings.get("inaturalist", True),
    description="📸 iNaturalist observations",
    style={"description_width": "200px"},
)

use_wikimedia_widget = widgets.Checkbox(
    value=default_settings.get("wikimedia", True),
    description="🎨 Wikimedia illustrations",
    style={"description_width": "200px"},
)

use_laji_widget = widgets.Checkbox(
    value=default_settings.get("laji", False),
    description="🇫🇮 Laji.fi Finnish names (requires API token)",
    style={"description_width": "200px"},
)

use_bhl_widget = widgets.Checkbox(
    value=default_settings.get("bhl", False),
    description="📚 BHL illustrations (requires API key)",
    style={"description_width": "200px"},
)

use_gemini_widget = widgets.Checkbox(
    value=default_settings.get("gemini", True),
    description="🤖 Gemini AI summary",
    style={"description_width": "200px"},
)

save_button = widgets.Button(
    description="💾 Save Settings",
    button_style="success",
    icon="check",
    layout=widgets.Layout(width="200px"),
)

status_output = widgets.Output()


def update_settings(_=None) -> None:
    """
    Updates global variables from UI values and checks API keys.

    Reads API keys securely from Colab Secrets.
    """
    global plant_name, output_language, TREFLE_API_KEY, GEMINI_API_KEY, BHL_API_KEY, LAJI_TOKEN, API_SETTINGS

    plant_name = plant_name_widget.value.strip()
    output_language = language_widget.value

    # Fetch API keys securely
    def get_api_key(key_name: str) -> str:
        """Fetches API key securely."""
        try:
            if COLAB_ENV:
                return userdata.get(key_name)
            else:
                return ""
        except Exception:
            return ""

    TREFLE_API_KEY = get_api_key("TREFLE_API_KEY")
    GEMINI_API_KEY = get_api_key("GOOGLE_API_KEY")
    BHL_API_KEY = get_api_key("BHL_API_KEY")
    LAJI_TOKEN = get_api_key("LAJI_TOKEN")

    # Update settings
    API_SETTINGS = {
        "gbif": use_gbif_widget.value,
        "trefle": use_trefle_widget.value,
        "eol": use_eol_widget.value,
        "gbif_occ": use_gbif_occ_widget.value,
        "gbif_dist": use_gbif_dist_widget.value,
        "wikipedia": use_wikipedia_widget.value,
        "inaturalist": use_inaturalist_widget.value,
        "wikimedia": use_wikimedia_widget.value,
        "laji": use_laji_widget.value,
        "bhl": use_bhl_widget.value,
        "gemini": use_gemini_widget.value,
    }

    # Get language name
    lang_names = {
        "en": "English",
        "fi": "Finnish",
        "sv": "Swedish",
        "de": "German",
        "fr": "French",
        "es": "Spanish",
        "it": "Italian",
        "ja": "Japanese",
        "zh": "Chinese",
    }

    # Show status
    with status_output:
        status_output.clear_output()
        if not plant_name:
            print("⚠️ Enter the plant's scientific name (e.g., Quercus robur)")
        else:
            print(f"✅ Settings updated: {plant_name}")
            print(
                f"🌍 Output language: {lang_names.get(output_language, output_language)}"
            )
            print("\n📋 API Key Status:")
            print(f"  Trefle API: {'✓ Set' if TREFLE_API_KEY else '✗ Missing'}")
            print(f"  Gemini API: {'✓ Set' if GEMINI_API_KEY else '✗ Missing'}")
            print(f"  BHL API: {'✓ Set' if BHL_API_KEY else '✗ Missing'}")
            print(f"  Laji.fi Token: {'✓ Set' if LAJI_TOKEN else '✗ Missing'}")

            # Warn about missing keys if sources are selected
            if API_SETTINGS.get("trefle") and not TREFLE_API_KEY:
                print("\n⚠️ Trefle selected but API key missing")
            if API_SETTINGS.get("gemini") and not GEMINI_API_KEY:
                print("⚠️ Gemini selected but API key missing")
            if API_SETTINGS.get("bhl") and not BHL_API_KEY:
                print("⚠️ BHL selected but API key missing")
            if API_SETTINGS.get("laji") and not LAJI_TOKEN:
                print("⚠️ Laji.fi selected but API token missing")


# Connect button to function
save_button.on_click(update_settings)

# Initialize settings
update_settings()

# Create UI
panel = widgets.VBox(
    [
        widgets.HTML("<h3>🌿 Plant Information</h3>"),
        plant_name_widget,
        language_widget,
        widgets.HTML(
            "<p><i>Language affects Wikipedia summaries and AI-generated content</i></p>"
        ),
        widgets.HTML("<h3>📚 Select Data Sources</h3>"),
        widgets.HTML(
            "<p><i>Check your desired data sources. For sources requiring API keys, add them to Colab Secrets.</i></p>"
        ),
        widgets.GridBox(
            [
                use_gbif_widget,
                use_trefle_widget,
                use_eol_widget,
                use_gbif_occ_widget,
                use_gbif_dist_widget,
                use_wikipedia_widget,
                use_inaturalist_widget,
                use_wikimedia_widget,
                use_laji_widget,
                use_bhl_widget,
                use_gemini_widget,
            ],
            layout=widgets.Layout(
                grid_template_columns="repeat(3, 33%)", grid_gap="10px"
            ),
        ),
        widgets.HTML("<br>"),
        widgets.HBox([save_button]),
        status_output,
    ],
    layout=widgets.Layout(
        padding="20px", border="2px solid #ddd", border_radius="10px"
    ),
)

display(panel)

## 🔧 Step 2: Helper Functions

These functions include error handling and data validation.

In [ ]:
# ============================================================================
# Helper Functions for Data Retrieval and Validation
# ============================================================================
"""
Contains general-purpose helper functions for API calls, error handling,
and data validation.
"""


def safe_api_call(
    url: str,
    params: Optional[Dict[str, Any]] = None,
    timeout: int = 15,
    method: str = "GET",
) -> Optional[Dict[str, Any]]:
    """
    Safe API call with error handling.

    Args:
        url: API endpoint
        params: Query parameters
        timeout: Timeout in seconds
        method: HTTP method (GET or POST)

    Returns:
        JSON response as dictionary or None on error
    """
    try:
        if method.upper() == "GET":
            response = requests.get(url, params=params, timeout=timeout)
        else:
            response = requests.post(url, json=params, timeout=timeout)

        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        print(f"⏱️ Timeout: {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error: {str(e)[:100]}")
        return None
    except ValueError:
        print("❌ Invalid JSON response")
        return None


def validate_scientific_name(name: str) -> bool:
    """
    Validates basic scientific name format.

    Args:
        name: Name to validate

    Returns:
        True if name is valid, False otherwise
    """
    if not name or len(name) < 3:
        return False

    # Scientific name contains a space (binomial nomenclature)
    parts = name.split()
    if len(parts) < 2:
        print("⚠️ Note: Scientific names are usually binomial (e.g., Quercus robur)")
        return True  # Allow continuing anyway

    # First letter uppercase, rest lowercase
    if not parts[0][0].isupper():
        print("⚠️ Note: Genus name usually starts with uppercase")

    return True


def format_data_for_display(
    data: Optional[Dict[str, Any]], source_name: str
) -> pd.DataFrame:
    """
    Formats data as Pandas DataFrame for display.

    Args:
        data: Data to display
        source_name: Data source name

    Returns:
        DataFrame for display
    """
    if not data:
        return pd.DataFrame(
            {"Field": [f"{source_name} data"], "Value": ["Not available"]}
        )

    rows = []
    for key, value in data.items():
        if value and value != "N/A":
            rows.append({"Field": key, "Value": str(value)})

    return (
        pd.DataFrame(rows)
        if rows
        else pd.DataFrame(
            {"Field": [f"{source_name} data"], "Value": ["Not available"]}
        )
    )


def create_citation(source: str, date: str = None) -> str:
    """
    Creates a citation for a data source.

    Args:
        source: Data source name
        date: Query date

    Returns:
        Formatted citation
    """
    if date is None:
        date = datetime.now().strftime("%Y-%m-%d")

    citations = {
        "GBIF": f"GBIF.org ({date}). GBIF Occurrence Download. https://doi.org/10.15468/dl.xxxxx",
        "Trefle": f"Trefle API ({date}). Plant data. https://trefle.io",
        "EOL": f"Encyclopedia of Life ({date}). https://eol.org",
        "Wikipedia": f"Wikipedia ({date}). https://wikipedia.org",
        "Wikimedia": f"Wikimedia Commons ({date}). https://commons.wikimedia.org",
        "iNaturalist": f"iNaturalist ({date}). https://www.inaturalist.org",
        "BHL": f"Biodiversity Heritage Library ({date}). https://biodiversitylibrary.org",
    }

    return citations.get(source, f"{source} ({date})")


print("✅ Helper functions defined")

## 📡 Step 3: Data Fetching Functions

These functions retrieve data from various APIs.

In [ ]:
# ============================================================================
# GBIF API Functions (Taxonomy and Distribution)
# ============================================================================
"""
Functions for retrieving taxonomy, occurrences, and distribution data from GBIF.

Data Sources:
- GBIF.org (2025) GBIF Backbone Taxonomy. https://doi.org/10.15468/39omei
"""


def fetch_gbif_data(scientific_name: str) -> Optional[Dict[str, Any]]:
    """
    Fetches taxonomy data from GBIF.

    Args:
        scientific_name: Scientific name of the plant (binomial)

    Returns:
        Dictionary with taxonomy data or None

    Source:
        GBIF.org - https://www.gbif.org/species/search
    """
    url = "https://api.gbif.org/v1/species/match"
    params = {"name": scientific_name}

    data = safe_api_call(url, params)
    if not data:
        return None

    result = {
        "Kingdom": data.get("kingdom", "N/A"),
        "Phylum": data.get("phylum", "N/A"),
        "Class": data.get("class", "N/A"),
        "Order": data.get("order", "N/A"),
        "Family": data.get("family", "N/A"),
        "Genus": data.get("genus", "N/A"),
        "Species": data.get("species", "N/A"),
        "Taxonomic Status": data.get("status", "N/A"),
        "Confidence": data.get("confidence", "N/A"),
        "GBIF ID": data.get("usageKey", "N/A"),
    }

    print("✅ GBIF: Taxonomy data retrieved")
    return result


def fetch_gbif_occurrences(scientific_name: str) -> Optional[Dict[str, Any]]:
    """
    Fetches occurrence count from GBIF.

    Args:
        scientific_name: Scientific name of the plant

    Returns:
        Dictionary with occurrence count or None
    """
    # First get taxon key
    match_url = "https://api.gbif.org/v1/species/match"
    match_data = safe_api_call(match_url, {"name": scientific_name})

    if not match_data or not match_data.get("usageKey"):
        print(f"❌ GBIF: No taxon key found for {scientific_name}")
        return None

    species_key = match_data["usageKey"]

    # Get occurrence count
    count_url = f"https://api.gbif.org/v1/occurrence/count?taxonKey={species_key}"
    count_data = safe_api_call(count_url)

    if count_data is not None:
        print("✅ GBIF: Occurrence data retrieved")
        return {"GBIF Occurrence Count": count_data}

    return None


def fetch_gbif_distribution(scientific_name: str) -> Optional[Dict[str, Any]]:
    """
    Fetches distribution data from GBIF.

    Args:
        scientific_name: Scientific name of the plant

    Returns:
        Dictionary with distribution summary or None
    """
    # First get taxon key
    match_url = "https://api.gbif.org/v1/species/match"
    match_data = safe_api_call(match_url, {"name": scientific_name})

    if not match_data or not match_data.get("usageKey"):
        return None

    species_key = match_data["usageKey"]

    # Get distributions
    dist_url = f"https://api.gbif.org/v1/species/{species_key}/distributions"
    dist_data = safe_api_call(dist_url)

    if not dist_data or not dist_data.get("results"):
        return None

    # Summarize distribution areas
    locations = []
    for dist in dist_data.get("results", [])[:10]:  # Limit to 10
        location = dist.get("locality") or dist.get("locationId", "Unknown")
        if location and location not in locations:
            locations.append(location)

    result = {
        "Distribution": (
            ", ".join(locations) if locations else "Available but no named areas"
        ),
        "Source Count": len(dist_data.get("results", [])),
    }

    print("✅ GBIF: Distribution data retrieved")
    return result


# ============================================================================
# Trefle API Functions (Plant Characteristics)
# ============================================================================
"""
Functions for retrieving growth characteristics and images from Trefle.

Data Source:
- Trefle.io - https://trefle.io
"""


def fetch_trefle_data(scientific_name: str, api_key: str) -> Optional[Dict[str, Any]]:
    """
    Fetches plant characteristics from Trefle API.

    Args:
        scientific_name: Scientific name of the plant
        api_key: Trefle API key

    Returns:
        Dictionary with plant data or None

    Source:
        Trefle.io - https://trefle.io
    """
    if not api_key:
        print("❌ Trefle: API key missing")
        return None

    url = "https://trefle.io/api/v1/plants/search"
    params = {"token": api_key, "q": scientific_name}

    data = safe_api_call(url, params)
    if not data or not data.get("data"):
        print(f"❌ Trefle: No data found for {scientific_name}")
        return None

    plant = data["data"][0]

    result = {
        "Common Name (English)": plant.get("common_name", "N/A"),
        "Growth Form": plant.get("main_species", {})
        .get("growth", {})
        .get("description", "N/A"),
        "pH Range": f"{plant.get('main_species', {}).get('growth', {}).get('ph_minimum', 'N/A')} - {plant.get('main_species', {}).get('growth', {}).get('ph_maximum', 'N/A')}",
        "Light Requirement": plant.get("main_species", {})
        .get("growth", {})
        .get("light", "N/A"),
        "Image URL (Trefle)": plant.get("image_url", "N/A"),
    }

    print("✅ Trefle: Plant data retrieved")
    return result


# ============================================================================
# EOL API Functions (Ecological Data)
# ============================================================================
"""
Functions for retrieving ecological information from Encyclopedia of Life.

Data Source:
- EOL.org - https://eol.org
"""


def fetch_eol_data(scientific_name: str) -> Optional[Dict[str, Any]]:
    """
    Fetches ecological data from EOL API.

    Args:
        scientific_name: Scientific name of the plant

    Returns:
        Dictionary with habitat and reproduction data or None

    Source:
        Encyclopedia of Life - https://eol.org
    """
    search_url = "http://eol.org/api/search/1.0.json"
    search_params = {"q": scientific_name, "page": 1, "exact": True, "cache_ttl": 60}

    search_data = safe_api_call(search_url, search_params)

    if not search_data or not search_data.get("results"):
        print(f"❌ EOL: No data found for {scientific_name}")
        return None

    # Get first result ID
    eol_id = search_data["results"][0]["id"]

    # Get detailed page data
    page_url = f"http://eol.org/api/pages/1.0/{eol_id}.json"
    page_params = {"cache_ttl": 60, "details": True, "common_names": True}

    page_data = safe_api_call(page_url, page_params)

    if not page_data:
        return None

    eol_result = {}

    # Extract habitat and reproduction information
    for obj in page_data.get("dataObjects", []):
        subject = obj.get("subject", "")
        if subject == "Habitat":
            eol_result["Habitat (EOL)"] = obj.get("description", "N/A")
        elif subject == "Reproduction":
            eol_result["Reproduction (EOL)"] = obj.get("description", "N/A")

    if eol_result:
        print("✅ EOL: Ecological data retrieved")
        return eol_result

    return None


print("✅ Data fetching functions defined (GBIF, Trefle, EOL)")

In [ ]:
# ============================================================================
# Wikipedia API Functions
# ============================================================================
"""
Functions for retrieving Wikipedia summaries and images.

Data Source:
- Wikipedia.org - https://wikipedia.org
License: CC-BY-SA
"""


def fetch_wikipedia_data(
    scientific_name: str, preferred_language: str = "en"
) -> Optional[Dict[str, Any]]:
    """
    Fetches summary from Wikipedia REST API in preferred language.

    Args:
        scientific_name: Scientific name of the plant
        preferred_language: Preferred language code (e.g., 'en', 'fi', 'de')

    Returns:
        Dictionary with Wikipedia data or None

    Source:
        Wikipedia REST API - https://en.wikipedia.org/api/rest_v1/
    """
    # Try preferred language first, then fallback to English
    languages = [preferred_language] if preferred_language != "en" else ["en"]
    if preferred_language != "en":
        languages.append("en")  # Add English as fallback

    for lang in languages:
        url = f"https://{lang}.wikipedia.org/api/rest_v1/page/summary/{scientific_name.replace(' ', '_')}"

        data = safe_api_call(url)
        if data and data.get("extract"):
            result = {
                "summary": data.get("extract", "N/A"),
                "short_description": data.get("description", "N/A"),
                "article_url": data.get("content_urls", {})
                .get("desktop", {})
                .get("page", "N/A"),
                "image_url": data.get("thumbnail", {}).get("source", "N/A"),
                "language": lang,
            }
            print(f"✅ Wikipedia: Summary retrieved ({lang})")
            return result

    print(f"❌ Wikipedia: No summary found for {scientific_name}")
    return None


# ============================================================================
# Laji.fi API Functions (Finnish Species Names)
# ============================================================================
"""
Functions for retrieving Finnish species names from Laji.fi.

Data Source:
- Laji.fi - https://laji.fi
"""


def fetch_laji_data(scientific_name: str, api_token: str) -> Optional[Dict[str, Any]]:
    """
    Fetches Finnish species names from Laji.fi API.

    Args:
        scientific_name: Scientific name of the plant
        api_token: Laji.fi API token

    Returns:
        Dictionary with Finnish name or None

    Source:
        Laji.fi - https://laji.fi
    """
    if not api_token:
        print("❌ Laji.fi: API token missing")
        return None

    url = "https://api.laji.fi/v0/taxa"
    params = {
        "scientificName": scientific_name,
        "access_token": api_token,
        "lang": "fi",
    }

    data = safe_api_call(url, params)

    if not data or not data.get("results"):
        print(f"❌ Laji.fi: No data found for {scientific_name}")
        return None

    # Get first result
    taxon = data["results"][0]

    result = {
        "Finnish Name": taxon.get("vernacularName", {}).get("fi", "N/A"),
        "Taxon ID": taxon.get("id", "N/A"),
        "Taxonomic Status": taxon.get("taxonomicStatus", "N/A"),
    }

    print("✅ Laji.fi: Finnish name retrieved")
    return result


# ============================================================================
# iNaturalist API Functions
# ============================================================================
"""
Functions for retrieving observation data from iNaturalist.

Data Source:
- iNaturalist.org - https://www.inaturalist.org
"""


def fetch_inaturalist_data(scientific_name: str) -> Optional[Dict[str, Any]]:
    """
    Fetches observation data from iNaturalist API.

    Args:
        scientific_name: Scientific name of the plant

    Returns:
        Dictionary with observation count and photos or None

    Source:
        iNaturalist - https://www.inaturalist.org
    """
    # First search for taxon
    search_url = "https://api.inaturalist.org/v1/taxa"
    search_params = {"q": scientific_name, "rank": "species"}

    search_data = safe_api_call(search_url, search_params)

    if not search_data or not search_data.get("results"):
        print(f"❌ iNaturalist: No taxon found for {scientific_name}")
        return None

    taxon = search_data["results"][0]
    taxon_id = taxon.get("id")

    if not taxon_id:
        return None

    # Get observations
    obs_url = "https://api.inaturalist.org/v1/observations"
    obs_params = {"taxon_id": taxon_id, "per_page": 1}

    obs_data = safe_api_call(obs_url, obs_params)

    result = {
        "observation_count": obs_data.get("total_results", 0) if obs_data else 0,
        "common_name": taxon.get("preferred_common_name", "N/A"),
        "page_url": f"https://www.inaturalist.org/taxa/{taxon_id}",
        "image_url": taxon.get("default_photo", {}).get("medium_url", "N/A"),
    }

    print("✅ iNaturalist: Observation data retrieved")
    return result


# ============================================================================
# Wikimedia Commons Functions
# ============================================================================
"""
Functions for retrieving botanical illustrations from Wikimedia Commons.

Data Source:
- Wikimedia Commons - https://commons.wikimedia.org
"""


def fetch_botanical_illustration(scientific_name: str) -> Optional[str]:
    """
    Fetches botanical illustration from Wikimedia Commons.

    Args:
        scientific_name: Scientific name of the plant

    Returns:
        Image URL or None

    Source:
        Wikimedia Commons - https://commons.wikimedia.org
    """
    base_url = "https://commons.wikimedia.org/w/api.php"

    # Try multiple search variations
    search_queries = [
        f"{scientific_name} botanical illustration",
        f"{scientific_name} botany",
        f"{scientific_name} illustration",
        scientific_name,
    ]

    fallback_image = None

    for query in search_queries:
        params = {
            "action": "query",
            "format": "json",
            "generator": "search",
            "gsrnamespace": "6",
            "gsrsearch": query,
            "gsrlimit": 5,
            "prop": "imageinfo",
            "iiprop": "url|mime|extmetadata",
            "iiurlwidth": 1200,
            "origin": "*",
        }

        data = safe_api_call(base_url, params)

        if not data or not data.get("query", {}).get("pages"):
            continue

        pages = data["query"]["pages"]

        for page in pages.values():
            for image_info in page.get("imageinfo", []):
                # Check if it's an image
                if not image_info.get("mime", "").startswith("image/"):
                    continue

                image_url = image_info.get("thumburl") or image_info.get("url")
                if not image_url:
                    continue

                # Check metadata for botanical content
                metadata = image_info.get("extmetadata", {})
                categories = metadata.get("Categories", {}).get("value", "").lower()
                description = (
                    metadata.get("ImageDescription", {}).get("value", "").lower()
                )

                # Prioritize botanical illustrations
                if (
                    any(
                        keyword in categories
                        for keyword in ["botanical illustration", "botany", "herbarium"]
                    )
                    or "illustration" in description
                ):
                    print(
                        f"✅ Wikimedia Commons: Botanical illustration found with query '{query}'"
                    )
                    return image_url

                # Keep first image as fallback
                if fallback_image is None:
                    fallback_image = image_url

    if fallback_image:
        print(
            "ℹ️ Wikimedia Commons: Image found but not confirmed botanical illustration. Returning best match."
        )
        return fallback_image

    print(
        f"❌ Wikimedia Commons: No botanical illustration found for {scientific_name}"
    )
    return None


# ============================================================================
# BHL API Functions (Historical Illustrations)
# ============================================================================
"""
Functions for retrieving historical botanical illustrations from BHL.

Data Source:
- Biodiversity Heritage Library - https://biodiversitylibrary.org
"""


def fetch_bhl_images(scientific_name: str, api_key: str) -> Optional[List[str]]:
    """
    Fetches botanical illustrations from Biodiversity Heritage Library.

    Args:
        scientific_name: Scientific name of the plant
        api_key: BHL API key

    Returns:
        List of image URLs or None

    Source:
        BHL - https://biodiversitylibrary.org
    """
    if not api_key:
        print("❌ BHL: API key missing")
        return None

    base_url = "https://www.biodiversitylibrary.org/api2/httpquery.ashx"

    # Search for books
    search_params = {
        "op": "BookSearch",
        "searchterm": scientific_name,
        "apikey": api_key,
        "format": "json",
    }

    search_data = safe_api_call(base_url, search_params)

    if (
        not search_data
        or search_data.get("Status") != "ok"
        or not search_data.get("Result")
    ):
        print(f"❌ BHL: No results found for {scientific_name}")
        return None

    item_id = search_data["Result"][0].get("ItemID")
    if not item_id:
        print(f"❌ BHL: No ItemID for {scientific_name}")
        return None

    # Get images for the item
    images_params = {
        "op": "GetItemImages",
        "itemid": item_id,
        "apikey": api_key,
        "format": "json",
    }

    images_data = safe_api_call(base_url, images_params)

    if (
        not images_data
        or images_data.get("Status") != "ok"
        or not images_data.get("Result")
    ):
        print("❌ BHL: No image data found")
        return None

    # Collect image URLs
    image_urls = []
    for page in images_data.get("Result", []):
        page_id = page.get("PageID")
        if page_id:
            image_url = f"https://www.biodiversitylibrary.org/pageimage/{page_id}"
            image_urls.append(image_url)
        if len(image_urls) >= 5:  # Limit to 5 images
            break

    if image_urls:
        print(f"✅ BHL: {len(image_urls)} images retrieved")
        return image_urls

    print("❌ BHL: No images found")
    return None


print(
    "✅ Additional fetching functions defined (Wikipedia, Laji.fi, iNaturalist, Wikimedia, BHL)"
)

## 🚀 Step 4: Execute Data Collection

Run this cell to fetch data from all selected sources.

In [ ]:
# ============================================================================
# Main Execution: Collect Data from All Sources
# ============================================================================
"""
Executes data collection from all selected APIs and compiles results.
"""

if not plant_name:
    print("❌ ERROR: Enter plant name in Step 1 and press 'Save Settings'")
else:
    # Validate plant name
    if not validate_scientific_name(plant_name):
        print(
            "⚠️ WARNING: Plant name may not be in correct format. Continuing anyway..."
        )

    lang_names = {
        "en": "English",
        "fi": "Finnish",
        "sv": "Swedish",
        "de": "German",
        "fr": "French",
        "es": "Spanish",
        "it": "Italian",
        "ja": "Japanese",
        "zh": "Chinese",
    }

    print(f"\n🌿 Collecting data for: {plant_name}")
    print(f"🌍 Output language: {lang_names.get(output_language, output_language)}")
    print("=" * 60)

    # Initialize result storage
    all_data = {}

    # 1. GBIF Taxonomy
    if API_SETTINGS.get("gbif", True):
        print("\n📍 Fetching GBIF taxonomy...")
        gbif_data = fetch_gbif_data(plant_name)
        if gbif_data:
            all_data.update(gbif_data)
    else:
        print("ℹ️ GBIF taxonomy skipped per settings")
        gbif_data = None

    # 2. Trefle Characteristics
    if API_SETTINGS.get("trefle", True):
        print("\n📍 Fetching Trefle data...")
        trefle_data = fetch_trefle_data(plant_name, TREFLE_API_KEY)
        if trefle_data:
            all_data.update(trefle_data)
    else:
        print("ℹ️ Trefle data skipped per settings")
        trefle_data = None

    # 3. EOL Ecological Data
    if API_SETTINGS.get("eol", True):
        print("\n📍 Fetching EOL data...")
        eol_data = fetch_eol_data(plant_name)
        if eol_data:
            all_data.update(eol_data)
    else:
        print("ℹ️ EOL data skipped per settings")
        eol_data = None

    # 4. GBIF Occurrences
    if API_SETTINGS.get("gbif_occ", True):
        print("\n📍 Fetching GBIF occurrences...")
        gbif_occ_data = fetch_gbif_occurrences(plant_name)
        if gbif_occ_data:
            all_data.update(gbif_occ_data)
    else:
        print("ℹ️ GBIF occurrences skipped per settings")
        gbif_occ_data = None

    # 5. GBIF Distribution
    if API_SETTINGS.get("gbif_dist", True):
        print("\n📍 Fetching GBIF distribution...")
        gbif_dist_data = fetch_gbif_distribution(plant_name)
        if gbif_dist_data:
            all_data.update(gbif_dist_data)
    else:
        print("ℹ️ GBIF distribution skipped per settings")
        gbif_dist_data = None

    # 6. Wikipedia Summary (in selected language)
    if API_SETTINGS.get("wikipedia", True):
        print(f"\n📍 Fetching Wikipedia summary ({output_language})...")
        wikipedia_data = fetch_wikipedia_data(plant_name, output_language)
        if wikipedia_data:
            # Don't add full summary to table, just metadata
            all_data["Wikipedia Language"] = wikipedia_data.get("language", "N/A")
            all_data["Wikipedia Short Description"] = wikipedia_data.get(
                "short_description", "N/A"
            )
    else:
        print("ℹ️ Wikipedia summary skipped per settings")
        wikipedia_data = None

    # 7. Laji.fi Finnish Names
    if API_SETTINGS.get("laji", False):
        print("\n📍 Fetching Laji.fi Finnish names...")
        laji_data = fetch_laji_data(plant_name, LAJI_TOKEN)
        if laji_data:
            all_data.update(laji_data)
    else:
        print("ℹ️ Laji.fi data skipped per settings")
        laji_data = None

    # 8. iNaturalist Observations
    if API_SETTINGS.get("inaturalist", True):
        print("\n📍 Fetching iNaturalist data...")
        inaturalist_data = fetch_inaturalist_data(plant_name)
        if inaturalist_data:
            all_data["iNaturalist Observations"] = inaturalist_data.get(
                "observation_count", "N/A"
            )
            all_data["iNaturalist Common Name"] = inaturalist_data.get(
                "common_name", "N/A"
            )
    else:
        print("ℹ️ iNaturalist data skipped per settings")
        inaturalist_data = None

    # 9. Wikimedia Commons Illustration
    if API_SETTINGS.get("wikimedia", True):
        print("\n📍 Fetching Wikimedia illustration...")
        illustration_url = fetch_botanical_illustration(plant_name)
    else:
        print("ℹ️ Wikimedia illustration skipped per settings")
        illustration_url = None

    # 10. BHL Historical Illustrations
    if API_SETTINGS.get("bhl", False):
        print("\n📍 Fetching BHL illustrations...")
        bhl_images = fetch_bhl_images(plant_name, BHL_API_KEY)
    else:
        print("ℹ️ BHL illustrations skipped per settings")
        bhl_images = None

    print("\n" + "=" * 60)
    print("✅ Data collection complete!")
    print("=" * 60)

    # Create DataFrame for display
    plant_df = pd.DataFrame(
        [
            {"Field": key, "Value": value}
            for key, value in all_data.items()
            if value and value != "N/A"
        ]
    )

    if plant_df.empty:
        plant_df = pd.DataFrame(
            [{"Field": "Status", "Value": "No data available from selected sources"}]
        )

## 📊 Step 5: Display Plant Card

Beautiful formatted display of all collected data.

In [ ]:
# ============================================================================
# Visualization: Display Plant Card with Data and Images
# ============================================================================
"""
Creates a beautifully formatted plant card with all collected information.
"""

# Display header
display(Markdown(f"# 🌲 PLANT CARD: **{plant_name}**"))
display(Markdown("***"))

# Display data table
if not plant_df.empty:
    display(Markdown("## 📋 Collected Information"))
    display(plant_df.style.hide(axis="index"))
else:
    display(Markdown("⚠️ No data could be retrieved from any source."))

# Display Wikipedia summary if available
if wikipedia_data and wikipedia_data.get("summary"):
    display(
        Markdown(
            f"\n## 📖 Wikipedia Summary ({wikipedia_data.get('language', 'en').upper()})"
        )
    )
    summary_text = wikipedia_data["summary"]
    if len(summary_text) > 500:
        summary_text = summary_text[:497] + "..."
    display(Markdown(summary_text))
    if wikipedia_data.get("article_url"):
        display(Markdown(f"[📚 Read full article]({wikipedia_data['article_url']})"))

# Display images
display(Markdown("\n## 🖼 Images and Illustrations"))

images_found = False

# Wikimedia botanical illustration
if illustration_url:
    images_found = True
    display(Markdown("### 🎨 Botanical Illustration (Wikimedia Commons)"))
    try:
        display(Image(url=illustration_url, width=600))
    except Exception as e:
        display(Markdown(f"> *Image display error: {str(e)}*"))
elif API_SETTINGS.get("wikimedia", True):
    display(Markdown("> *No botanical illustration found on Wikimedia Commons.*"))

# Trefle photo
trefle_image_url = trefle_data.get("Image URL (Trefle)") if trefle_data else None
if trefle_image_url and trefle_image_url != "N/A":
    images_found = True
    display(Markdown("### 📸 Photograph (Trefle API)"))
    try:
        display(Image(url=trefle_image_url, width=600))
    except Exception as e:
        display(Markdown(f"> *Image display error: {str(e)}*"))

# Wikipedia image
wiki_image_url = wikipedia_data.get("image_url") if wikipedia_data else None
if wiki_image_url and wiki_image_url != "N/A":
    images_found = True
    display(Markdown("### 📘 Image (Wikipedia)"))
    try:
        display(Image(url=wiki_image_url, width=600))
    except Exception as e:
        display(Markdown(f"> *Image display error: {str(e)}*"))

# iNaturalist observation photo
inaturalist_image_url = inaturalist_data.get("image_url") if inaturalist_data else None
if inaturalist_image_url and inaturalist_image_url != "N/A":
    images_found = True
    display(Markdown("### 🌍 Observation Photo (iNaturalist)"))
    try:
        display(Image(url=inaturalist_image_url, width=600))
    except Exception as e:
        display(Markdown(f"> *Image display error: {str(e)}*"))

# BHL historical illustrations
if bhl_images:
    images_found = True
    display(Markdown("### 📚 Historical Illustrations (Biodiversity Heritage Library)"))
    for idx, img_url in enumerate(bhl_images[:3], 1):  # Limit to 3 for display
        try:
            display(Image(url=img_url, width=600))
        except Exception as e:
            display(Markdown(f"> *Image {idx} display error: {str(e)}*"))

if not images_found:
    display(Markdown("> *No images available from selected sources.*"))

# Display links
display(Markdown("\n## 🔗 External Links"))
links = []

if inaturalist_data and inaturalist_data.get("page_url"):
    links.append(f"- [iNaturalist Page]({inaturalist_data['page_url']})")
if wikipedia_data and wikipedia_data.get("article_url"):
    links.append(
        f"- [Wikipedia Article ({wikipedia_data.get('language', 'en').upper()})]({wikipedia_data['article_url']})"
    )
if gbif_data and gbif_data.get("GBIF ID") != "N/A":
    links.append(
        f"- [GBIF Species Page](https://www.gbif.org/species/{gbif_data['GBIF ID']})"
    )

if links:
    display(Markdown("\n".join(links)))
else:
    display(Markdown("> *No external links available.*"))

# Display data sources and citations
display(Markdown("\n## 📚 Data Sources & Citations"))
display(Markdown(f"Data retrieved on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"))

citations = []
if gbif_data:
    citations.append(create_citation("GBIF"))
if trefle_data:
    citations.append(create_citation("Trefle"))
if eol_data:
    citations.append(create_citation("EOL"))
if wikipedia_data:
    citations.append(create_citation("Wikipedia"))
if inaturalist_data:
    citations.append(create_citation("iNaturalist"))
if illustration_url:
    citations.append(create_citation("Wikimedia"))
if bhl_images:
    citations.append(create_citation("BHL"))

for citation in citations:
    display(Markdown(f"- {citation}"))

print("\n✅ Plant card displayed successfully!")

## 🤖 Step 6: AI-Generated Summary (Optional)

Generate a comprehensive plant summary using Google Gemini AI.

In [ ]:
# ============================================================================
# AI Summary Generation with Google Gemini
# ============================================================================
"""
Generates a comprehensive, readable summary using Google Gemini AI in selected language.

⚠️ NOTE: AI-generated content is indicative and not peer-reviewed.
Always verify information from primary scientific sources.
"""

if not API_SETTINGS.get("gemini", True):
    print("ℹ️ Gemini AI summary skipped per settings")
elif not GEMINI_API_KEY:
    print("❌ Gemini API key missing. Add GOOGLE_API_KEY to Colab Secrets.")
    display(
        Markdown(
            """
    ### 🔑 How to Add API Key:
    1. Click the **🔑 key icon** in the left sidebar
    2. Add secret named: `GOOGLE_API_KEY`
    3. Paste your Gemini API key
    4. Re-run this cell
    
    Get your API key at: [Google AI Studio](https://aistudio.google.com/app/apikey)
    """
        )
    )
else:
    try:
        # Configure Gemini
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel("gemini-pro")

        # Language-specific prompts
        language_instructions = {
            "en": "Write the summary in English.",
            "fi": "Kirjoita yhteenveto suomeksi. (Write the summary in Finnish.)",
            "sv": "Skriv sammanfattningen på svenska. (Write the summary in Swedish.)",
            "de": "Schreiben Sie die Zusammenfassung auf Deutsch. (Write the summary in German.)",
            "fr": "Rédigez le résumé en français. (Write the summary in French.)",
            "es": "Escribe el resumen en español. (Write the summary in Spanish.)",
            "it": "Scrivi il riassunto in italiano. (Write the summary in Italian.)",
            "ja": "要約を日本語で書いてください。(Write the summary in Japanese.)",
            "zh": "用中文写摘要。(Write the summary in Chinese.)",
        }

        lang_instruction = language_instructions.get(
            output_language, "Write the summary in English."
        )

        # Build prompt from collected data
        prompt = f"""Create a concise, scientifically accurate summary of this plant based on the following data.
{lang_instruction}

Scientific Name: {plant_name}

"""

        # Add all available data to prompt
        if gbif_data:
            prompt += f"Kingdom: {gbif_data.get('Kingdom', 'N/A')}\n"
            prompt += f"Family: {gbif_data.get('Family', 'N/A')}\n"
            prompt += f"Genus: {gbif_data.get('Genus', 'N/A')}\n"

        if trefle_data:
            prompt += (
                f"Common Name: {trefle_data.get('Common Name (English)', 'N/A')}\n"
            )
            prompt += f"Growth Form: {trefle_data.get('Growth Form', 'N/A')}\n"
            prompt += f"pH Range: {trefle_data.get('pH Range', 'N/A')}\n"
            prompt += (
                f"Light Requirement: {trefle_data.get('Light Requirement', 'N/A')}\n"
            )

        if laji_data:
            prompt += f"Finnish Name: {laji_data.get('Finnish Name', 'N/A')}\n"

        if gbif_occ_data:
            prompt += f"GBIF Occurrences: {gbif_occ_data.get('GBIF Occurrence Count', 'N/A')}\n"

        if gbif_dist_data:
            prompt += f"Distribution: {gbif_dist_data.get('Distribution', 'N/A')}\n"

        if eol_data:
            if eol_data.get("Habitat (EOL)"):
                prompt += f"Habitat: {eol_data.get('Habitat (EOL)')[:200]}...\n"
            if eol_data.get("Reproduction (EOL)"):
                prompt += (
                    f"Reproduction: {eol_data.get('Reproduction (EOL)')[:200]}...\n"
                )

        if inaturalist_data:
            prompt += f"iNaturalist Observations: {inaturalist_data.get('observation_count', 'N/A')}\n"

        if wikipedia_data and wikipedia_data.get("summary"):
            prompt += f"\nWikipedia Summary: {wikipedia_data['summary'][:300]}...\n"

        prompt += f"""

Please provide (in the requested language):
1. A brief introduction (2-3 sentences)
2. Key characteristics
3. Habitat and distribution
4. Ecological significance or interesting facts

Keep the summary informative but accessible, around 200-250 words.
Remember: {lang_instruction}
"""

        # Generate summary
        lang_names = {
            "en": "English",
            "fi": "Finnish",
            "sv": "Swedish",
            "de": "German",
            "fr": "French",
            "es": "Spanish",
            "it": "Italian",
            "ja": "Japanese",
            "zh": "Chinese",
        }
        print(
            f"🤖 Generating AI summary with Gemini ({lang_names.get(output_language, output_language)})..."
        )
        response = model.generate_content(prompt)

        if response and response.text:
            display(
                Markdown(
                    f"## 🤖 AI-Generated Summary ({lang_names.get(output_language, output_language)})"
                )
            )
            display(Markdown(response.text))
            display(
                Markdown(
                    """
---
*⚠️ This summary was generated by AI and should be verified against primary scientific sources.*
            """
                )
            )
            print("✅ AI summary generated successfully")
        else:
            print("❌ Gemini returned empty response")

    except Exception as e:
        print(f"❌ Error generating AI summary: {str(e)}")
        display(
            Markdown(
                f"""
        ### ⚠️ AI Summary Error
        
        Could not generate summary: `{str(e)}`
        
        **Common issues:**
        - API key invalid or expired
        - API quota exceeded
        - Network connectivity issues
        
        Check your API key at: [Google AI Studio](https://aistudio.google.com/app/apikey)
        """
            )
        )

---

## 📝 Notes and Best Practices

### ✅ Data Quality
- Always verify scientific information from multiple sources
- Check data freshness - API data is dynamic
- Cross-reference AI summaries with peer-reviewed literature

### 🔬 Scientific Use
- Cite all data sources in publications
- Note API version and query date
- Be aware of taxonomic changes and synonyms

### 🐛 Troubleshooting

**No data returned:**
- Check scientific name spelling
- Verify API keys in Colab Secrets
- Try broader search (e.g., genus only)

**Images not displaying:**
- Check image URL validity
- Try different data sources
- Some species may lack photographs

**API errors:**
- Check API key validity
- Verify rate limits not exceeded
- Check network connectivity

### 🔗 Useful Resources
- [GBIF](https://www.gbif.org) - Global biodiversity data
- [Trefle API Docs](https://docs.trefle.io) - Plant database documentation
- [Google AI Studio](https://aistudio.google.com) - Get Gemini API key
- [BHL](https://www.biodiversitylibrary.org) - Historical botanical literature

---

## 📄 License & Attribution

**Code License:** MIT License

**Data Sources:** Each data source has its own license:
- GBIF: CC0 / CC-BY
- Wikipedia: CC-BY-SA
- Wikimedia Commons: Varies (mostly CC-BY or Public Domain)
- Trefle: Terms of Service apply
- EOL: Varies by content
- iNaturalist: Varies by user
- BHL: Public Domain (most content)

**Citation:** When using this notebook, please cite:
```
Sihvonen, P. (2025). Plant Card Generator v1.1. 
GitHub repository: https://github.com/[your-repo]/botanical-colab-notebooks
```

---

**Version:** 1.1  
**Last Updated:** 2025-11-04  
**Created by:** Pekka Sihvonen  
**Language:** English

🌲 Happy botanizing! 🌿